In [1]:
import boto3
import json
import os

In [2]:
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime', 
    aws_access_key_id=os.getenv('aws_access_key_id_3'),
    aws_secret_access_key=os.getenv('aws_secret_access_key_3'),
    region_name='us-west-2'
)

In [13]:
inference_modifier = {
    "max_tokens": 4096,  # Updated max tokens
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

In [11]:
def textgen_llm(prompt, model_kwargs={}):
    payload = {
        "anthropic_version": "bedrock-2023-05-31",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
    payload.update(model_kwargs)
    
    try:
        response = bedrock_runtime.invoke_model(
            modelId='anthropic.claude-3-5-sonnet-20241022-v2:0',  # Use the model ID that supports the Messages API
            body=json.dumps(payload).encode('utf-8'),
            contentType='application/json',
            accept='application/json'
        )
    except Exception as e:
        print("Error invoking the model:", e)
        return None

    # Read and decode the response
    result = response['body'].read().decode('utf-8')
    
    # Attempt to parse the result as JSON; if it fails, return the raw string.
    try:
        return json.loads(result)
    except json.JSONDecodeError:
        return result

## Zero-Shot Prompt

In [12]:
response = textgen_llm("""

Human: Develop a marketing strategy outline for launching a new eco-friendly,
innovative bicycle model targeted at urban commuters.

Assistant:""", model_kwargs=inference_modifier)
print(response)

{'id': 'msg_bdrk_01Qvzi5xXEGYsGVwtpY23E5u', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-5-sonnet-20241022', 'content': [{'type': 'text', 'text': "Here's a comprehensive marketing strategy outline for launching an eco-friendly bicycle model:\n\n1. Market Analysis\n- Target audience demographics: Urban professionals, 25-45 years old\n- Competitor analysis: Existing bike brands and alternative transport options\n- Market trends: Growing interest in sustainability, urban mobility solutions\n- Market size and growth potential\n\n2. Product Positioning\n- Unique Value Proposition: Eco-friendly materials + innovative design + urban practicality\n- Key differentiators: Sustainable manufacturing, smart features, urban-specific design\n- Brand personality: Modern, environmentally conscious, tech-savvy\n\n3. Marketing Mix\nPrice:\n- Premium pricing strategy reflecting quality and innovation\n- Special launch pricing and early-bird offers\n- Financing options for customers\n\nProduc

## One-Shot Prompt

In [5]:
response = textgen_llm("""

Human: I'm planning a dinner party for six people, and I'd like to serve an Italian-themed menu. Can you suggest a three-course meal?

Example: For a French-themed dinner party, you could serve a cheese platter as a starter, coq au vin for the main course, and crème brûlée for dessert.

Now, can you suggest a similar menu for the Italian theme?
Assistant:""", model_kwargs=inference_modifier)

print(response)

{'id': 'msg_bdrk_01JvqRW35s3VfcUwxZpuueHB', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-5-sonnet-20241022', 'content': [{'type': 'text', 'text': "For an Italian-themed dinner party of six people, here's a classic three-course menu:\n\nAntipasto (Starter):\n- Bruschetta al Pomodoro (toasted bread with fresh tomatoes, garlic, and basil)\n- OR an antipasto platter with prosciutto, mozzarella, olives, and marinated vegetables\n\nSecondo (Main Course):\n- Osso Buco alla Milanese (braised veal shanks) with saffron risotto\n- OR homemade fettuccine with classic Bolognese sauce\n\nDolce (Dessert):\n- Tiramisu (traditional coffee-flavored dessert with mascarpone)\n- OR Panna Cotta with fresh berries\n\nEach course represents different regions of Italy while remaining approachable for home cooking. Would you like the recipe for any of these dishes?"}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 102, 'cache_creation_input_tokens': 0, 'cache_read_inp

In [ ]:
##Prompt Template

In [7]:
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["marketingManager", "productType", "targetAudience", "productFeatures"],
    template="""\
Human: Draft a briefing for the marketing team led by {marketingManager}, focusing on the upcoming launch of our new {productType} designed for {targetAudience}. The briefing should highlight the key features of the product:
<product_features>
{productFeatures}
</product_features>
and emphasize the importance of aligning the marketing strategy with these features.

Example 1: 
Human: Draft a briefing for the marketing team led by Sarah, focusing on the upcoming launch of our new eco-friendly water bottle designed for outdoor enthusiasts. The briefing should highlight the key features of the product: 
<product_features>Reusable, insulated, lightweight</product_features> 
and emphasize the importance of aligning the marketing strategy with these features.

Assistant: The product's lightweight, insulated design is a key selling point, especially for outdoor enthusiasts...

Example 2:
Human: Draft a briefing for the marketing team led by John, focusing on the launch of a new smartwatch designed for tech-savvy consumers. Highlight the features:
<product_features>Fitness tracking, long battery life, sleek design</product_features>...

Now draft a similar briefing for Alex and the eco-friendly bicycle.

Assistant:"""
)

prompt = multi_var_prompt.format(
    marketingManager="Alex",
    productType="eco-friendly bicycle",
    targetAudience="urban commuters",
    productFeatures="""The bicycle features an innovative design with integrated smart technology for navigation and fitness tracking, a lightweight yet durable frame made from recycled materials, and a sleek, modern aesthetic appealing to city dwellers. It also includes enhanced safety features such as built-in lights and reflective surfaces for nighttime visibility."""
)

print(prompt)


Human: Draft a briefing for the marketing team led by Alex, focusing on the upcoming launch of our new eco-friendly bicycle designed for urban commuters. The briefing should highlight the key features of the product:
<product_features>
The bicycle features an innovative design with integrated smart technology for navigation and fitness tracking, a lightweight yet durable frame made from recycled materials, and a sleek, modern aesthetic appealing to city dwellers. It also includes enhanced safety features such as built-in lights and reflective surfaces for nighttime visibility.
</product_features>
and emphasize the importance of aligning the marketing strategy with these features.

Example 1: 
Human: Draft a briefing for the marketing team led by Sarah, focusing on the upcoming launch of our new eco-friendly water bottle designed for outdoor enthusiasts. The briefing should highlight the key features of the product: 
<product_features>Reusable, insulated, lightweight</product_features> 

In [8]:
response = textgen_llm(prompt)

print(response)

{'id': 'msg_bdrk_01MYKPUUAbkeLbiswHKhtnaZ', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-5-sonnet-20241022', 'content': [{'type': 'text', 'text': "MARKETING TEAM BRIEFING\nAttention: Alex and Team\nRe: Launch Strategy for Urban Eco-Commuter Bicycle\n\nDear Alex and team,\n\nI'm reaching out to brief you on our upcoming launch of the new eco-friendly urban commuter bicycle. This product represents a significant opportunity to capture the growing market of environmentally conscious urban commuters.\n\nKey Product Features to Emphasize:\n\n1. Smart Technology Integration\n- Built-in navigation system\n- Fitness tracking capabilities\n- Perfect for tech-savvy urban professionals\n\n2. Sustainable Construction\n- Frame constructed from recycled materials\n- Eco-friendly manufacturing process\n- Appeals to environmentally conscious consumers\n\n3. Safety & Design\n- Integrated lighting system\n- Reflective surfaces for night visibility\n- Modern, sleek aesthetic\n\nMarketing St